In [ ]:
import selenium 
from selenium import webdriver
import random
from time import sleep
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from tqdm import tqdm

In [ ]:
# Browser 켜기
browser = webdriver.Chrome()
browser.get('https://everytime.kr/timetable')

In [ ]:
# login 함수 생성
def login(id_,password):
    browser.find_element_by_name('userid').send_keys(str(id_))
    browser.find_element_by_name('password').send_keys(str(password))
    browser.find_element_by_xpath('//*[@id="container"]/form/p[3]/input').click()

In [ ]:
# 가끔 팝업뜨는거 닫아주기
browser.find_element_by_xpath('//*[@id="container"]/ul/li[1]').click()

In [ ]:
# 스크롤 위치 찾기
scr1=browser.find_element_by_xpath('//*[@id="subjects"]/div[2]')

In [ ]:
# 스크롤 계속 내리기 ( 매우 오래걸림 실행 X)
for i in range(1000):
    browser.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scr1)
    sleep(3)

In [ ]:
# 에타 과목 정보 저장
tables = browser.find_elements_by_tag_name("tr")

In [ ]:
# 이 과목 평점 저장
stars = browser.find_elements_by_class_name('star')

In [ ]:
# 칼럼 이름이 html에 있길래 뽑은건데, 2번씩 중복해서 나오길래 이렇게 뽑아봄 (사용안했음 ㅎㅎ;;ㅈㅅ;;)
cols=[]

for i in range(0,24,2):
    cols.append(table[2].text.split('\n')[i])

In [ ]:
# 점수랑 타이틀 가져오는 코드
scores=[]
links=[]

for table in tqdm (stars) :
    scores.append(table.get_attribute('title'))
    links.append(table.get_attribute('href'))

In [ ]:
# 테이블이, 3번행부터 정보가 있고, 2918행 이후로는 의미없는 행이길래 제외
# 테이블에서 세부정보 뽑는 코드
tables=tables[3:2918]
grades=[] # 추천학년
categories=[] # 수업 분류(전공,교양)
codes=[] # 과목코드
names=[] # 수업 이름
profs=[] # 교수 이름
weights=[] # 학점(1,2,3)
times=[] # 시간
competitiors=[] # 에타 시간표에 담은인원

for element in tqdm(tables):
    splited=element.text.split(' ')
    grades.append(splited[0])
    categories.append(splited[1])
    codes.append(splited[2])
    names.append(splited[3])
    profs.append(splited[4])
    weights.append(splited[5])
    try :
        times.append(splited[8]+splited[9]+splited[10]+splited[11]+splited[12])
    except :
        times.append(None)
    try :
        competitiors.append(splited[13])
    except :
        competitiors.append(None)

In [ ]:
# 뽑은 리스트들을 df로 바꿔줬음
dfs=[names,codes,scores,categories,profs,times,grades,weights,competitiors,links]
df=pd.DataFrame()
for cols in dfs:
    df=pd.concat([df,pd.Series(cols)],axis=1)

In [ ]:
# 칼럼 이름 만들어줌
df.columns=['class_name','class_code','score','category','professor','time',
            'recommend_year','weight','competitor','link']

In [ ]:
# 아까 위에서 형식 제대로 안지켜진 데이터들 떨굼 ( 보완 방향 (!) )
df.dropna(inplace=True)

In [ ]:
# 교수이름이 3글자 아닌 수업들 뽑아봄 : 왜냐면 그냥 인덱싱으로 한거라 형식 안지켜진건데 들어갔을 수 있음
error_list=[]
for i in df['professor'].unique():
    if len(i) !=3 :
        error_list.append(i)

In [ ]:
# 직접 보고 이름 2,4글자인 교수님들 이름 빼줌
non_error=['박진','이화','박민','호티롱안','나강','문영','전긍','고댕','김예솔란','정철',
                 '최웅','안준','김강','이건','사공석진','허림','오정','이민','이찬',
                 '김명','허균','김영','이웅','임강','최영']

In [ ]:
# 제거
for i in non_error :
    error_list.remove(i)

In [ ]:
# 그냥 이름3자 아니거나, non error 아닌 거 다 없애버렸는데 이거 보완해야함 ( 1 )
#df2=df.query('professor not in @error_list') 

In [ ]:
# Save
#df2.to_csv('df2.csv',index=False)
# Load
df2=pd.read_csv('df2.csv')

In [ ]:
# 링크 추출함, 리뷰 하나도 없는 수업은 java스크립트로 뽑혀오길래 그것도 제외
link_list=list(df2['link'].unique())
link_list.remove('javascript: alert("%EC%95%84%EC%A7%81 %EB%93%B1%EB%A1%9D%EB%90%9C %EA%B0%95%EC%9D%98%ED%8F%89%EC%9D%B4 %EC%97%86%EC%8A%B5%EB%8B%88%EB%8B%A4.");')

In [ ]:
from random import randint
from bs4 import BeautifulSoup as bs
import requests
import lxml.html
import re

In [ ]:
def review_crawl(link_text):
    ## Selenium
    browser.get(link_text)
    sleep(0.5) # 로딩 기다리기
    # 리뷰 내용만 추출
    reg=re.compile('[0-9]')
    reviews={}
    sleep(2) # 로딩 기다리기 
    tb=browser.find_element_by_class_name('articles')
    art_table=tb.find_elements_by_tag_name('article')

    for element in art_table :
        reviews[element.find_element_by_class_name('text').text]=(float(''.join(reg.findall(element.find_element_by_class_name('on').get_attribute('style'))))*0.05)

    # 소스 뺴와서 파싱
    table=bs(browser.page_source,'xml')
    heads=table.find('div',class_='side head')
    
    # 제목, 교수 내용 추출
    class_name=(heads.find('h2').text)
    prof_name=(heads.find('span').text)

    # 강의평 테이블 추출
    articles=table.find('div',class_='side article')

    # 강의평의 평균 평점 내용 추출
    mean_score=float(articles.find('span',class_='value').text)

    # 강의평 중 세부 테이블 추출
    details=articles.find('div',class_='details')

    # 강의평 중 세부 내용 추출
    labels=details.find_all('label')
    detail=details.find_all('span')
    detail_type={}
    for l,d in zip(labels,detail):
        detail_type[str(l.text)]=d.text

    review_t=pd.DataFrame.from_dict(reviews, orient='index').reset_index()
    review_t.columns=['review','score']
    review_t['name']=class_name
    review_t['prof']=prof_name
    
    review_detail=pd.DataFrame.from_dict(detail_type,orient='index').T
    review_detail['name']=class_name
    review_detail['prof']=prof_name

    return review_t,review_detail

In [ ]:
review=pd.DataFrame() # 강의평 추출 테이블
detail=pd.DataFrame() # 과목 세부정보 추출 테이블

In [ ]:
# ↓ 가끔 안 켜고 시작하면 오류뜰 때가 있어서 그때 대비 
browser = webdriver.Chrome()
browser.get(link_list[0]) 
login('###','####')
# ↑ 가끔 안 켜고 시작하면 오류뜰 때가 있어서 그때 대비

In [ ]:
# 크롤링 과정 (오래걸림 실행 ㄴ)
for link in tqdm(link_list) :
    try:
        r_t,r_d=review_crawl(link)
        review=pd.concat([review,r_t])
        detail=pd.concat([detail,r_d])
        sleep(1)
    except:
        print(link , '강의평 추출 X ')

In [ ]:
#Save
#review.to_csv('review.csv',index=False)
#detail.to_csv('detail.csv',index=False)
#Load
review=pd.read_csv('review.csv')
review=review.reset_index().drop('index',axis=1)

In [ ]:
review.head()

In [ ]:
def cleanText(readData):
    text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', ' ', readData) # 특문제거
    text = re.sub('[^가-힣]',' ',text) # 한글 제외, + 단일 자음모음 제거 
    return text

In [ ]:
clean=[]

for data in review['review']:
    clean.append(cleanText(data))

In [ ]:
review['review']=pd.Series(clean)

In [ ]:
from konlpy.tag import *
kkma = Kkma()

In [ ]:
# 토크나이징하고, 에러난 row ( 대부분 빈 텍스트를 토크나이징 한 것 ) 찾아줌
corpus=[] 
error=[]
i=0
for data in tqdm(review['review']):
    try:
        corpus.append(kkma.morphs(data))
    except:
        error.append(i)
    i += 1
error=[2568,2720,6009,6532,6589,7729,8722,10853]

In [ ]:
# 에러난 친구들
display(review.loc[error,:])
review=review.drop(error,0) # 빈 행 삭제

In [ ]:
# save
with open('kkma_token_0909.pickle', 'wb') as f:
    pickle.dump(corpus, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
# load
import pickle
with open('kkma_token_0909.pickle', 'rb') as f:
    corpus = pickle.load(f)

In [ ]:
from gensim.models import Word2Vec, FastText

embd_SG= Word2Vec(corpus, size=100, window=2, min_count=10,
                           workers=-1, iter=1000, sg=1)

In [ ]:
#CBOW보단 Skip - Gram의 성능이 전반적으로 낫다고들 함
#리뷰 여러개니까, 차원 적으면 표현이 안될거라고 생각, 100차원정도 임시로 줘봤음
#리뷰다보니까 별로 안 길어서 윈도우는 작게주고, min_count는 의미없는 단어 나오지 말라고 10정도로 줌
# -> 이거 나중에 보완 ( 2 )
# 리뷰 단위별 벡터구하기 (받은 코드가 옛날버전인듯 word2vec 모델에 그런인자가 없던데..그래서 그냥 짬)
def get_embedding_vector(model,corpus):
    embeding_dict={}
    embeding_df=pd.DataFrame()
    for w,v in zip(model.wv.index2word , model.wv.vectors):
        embeding_dict[w]=v
        embeding_table=(pd.DataFrame(embeding_dict).T.reset_index())
    for words in (corpus):
        embeding_df=pd.concat([embeding_df,
                               embeding_table.query('@words in index').iloc[:,1:].mean()]
                             ,axis=1 )
    return embeding_df

In [ ]:
%%time

#임베딩 벡터 추출
embedding_vect=get_embedding_vector(embd_SG,corpus)

In [ ]:
vect=(embedding_vect.T.reset_index()).iloc[:,1:] # 데이터 프레임으로 바꿔주고
vect.columns=['Review_Embeded'+str(x) for x in range(100)] # 칼럼이름도 달아줌

In [ ]:
# 리뷰랑 병합(Group화 해서 강의별 계산을 위함)
review.index=range(len(review))
df3=pd.concat([review.iloc[:,1:],vect],axis=1)

In [ ]:
# 여기다 담아놓은다음에 Merge
feature=[]

for col_name in (['Review_Embeded'+str(x) for x in range(100)]):
    feature.append(df3.groupby(['name','prof'])[col_name].mean().reset_index())

In [ ]:
# Merge의 과정
df=df2.drop(['class_code','time','link','competitor'],axis=1)
df.rename({'class_name':'name','professor':'prof'},axis=1,inplace=True)

In [ ]:
for f in feature :
    df =pd.merge(df, f, on=['name','prof'],how='left')

In [ ]:
df.drop_duplicates(inplace=True)
df.dropna(inplace=True)

In [ ]:
# Skip-Gram을 이용한 인베딩 테이블 완성
sim_df=pd.concat([df[['name','prof',]],df.iloc[:,6:]],axis=1)
sim_df.index=[x for x in range(892)]

In [ ]:
# 나중에 사용할 테이블
detail_table=df2[['class_name','professor','category','recommend_year','weight','score']].drop_duplicates()
detail_table.rename({'class_name':'name','professor':'prof'},axis=1,inplace=True)

In [ ]:
from numpy import dot
from numpy.linalg import norm
import numpy as np

In [ ]:
#코사인 유사도 계산 함수
def cos_sim(A, B): 
       return dot(A, B)/(norm(A)*norm(B))

In [ ]:
#다른 모든 행과 유사도 계산 함수
def find_similar(input_name,input_prof,df):
    input_lecture=df.query('name == @input_name & prof == @input_prof')
    rest_lecture=df.query('name != @input_name | prof != @input_prof')
    output=rest_lecture.iloc[:,:2]
    output['similar']=''
    for i in range(len(rest_lecture)):
        output.iloc[i,2]= cos_sim(input_lecture.iloc[:,2:],rest_lecture.iloc[i,2:])
    output = pd.merge(output, detail_table, on=['name','prof'],how='left')
    output = output.sort_values('similar',ascending=False)
    return output

In [ ]:
# 내가 들었던 수업중에 교수님이 (사람)좋았던 수업
find_similar('문명과세계화의도전','안현상',sim_df)[:5] # 경제학개론 확인

In [ ]:
# 강의는 썩 재미없었지만 교수님이 열정 넘치고 엄청 좋은 분이었음, 유은나 교수님의 경제학 개론도 대체적으로 비슷한 평가
display(review.query('name=="문명과세계화의도전" & prof =="안현상"')[['review']][:5])
display(review.query('name=="경제학개론" & prof =="유은나"')[['review']][:5])

In [ ]:
# 외우기만 해서 맘에 안들었던 수업, 영화시리즈 제외하고 한문과 문화 확인
display(find_similar('영화작문','이지현',sim_df)[:5]) 

In [ ]:
# 암기라는 키워드 자주등장, 차이점은 교수님 칭찬이 많음 ㅎ; 평점차인듯
display(review.query('name=="영화작문" & prof =="이지현"')[['review']][:5])
display(review.query('name=="한문과문화" & prof =="이규일"')[['review']][:5]) 

In [ ]:
from gensim.models import FastText
%%time

model = FastText.load_fasttext_format('D:/fasttext/wiki.ko.bin')
# Pre-triainde vector를 사용해보고 싶은데, 커뮤니티 리뷰다 보니까 별 이상한 단어랑 줄임말,오타들이 다있음
# -> 그래서 일반적 word2vec모델에선 사용이 안되것 같음
# -> 근데 FastText는 corpus에 없는 단어들도 사용 가능 + Noise에 강한모델
# -> 따라서 FastText를 이용한 Pre_Trained Vector 사용 시도

In [ ]:
fatx_vec=[] # 문장별 임베딩

for words in corpus:
    fatx_vec.append(model[words].mean(axis=0))

In [ ]:
fatx_emd=pd.DataFrame() # 임베딩한 벡터를 데이터프레임으로 만들어줌
for vecs in (fatx_vec):
    fatx_emd=pd.concat([fatx_emd,pd.DataFrame(vecs)],axis=1)

In [ ]:
# 벡터 데이터프레임 전처리    
fatx_emd=fatx_emd.T
fatx_emd.index=range(len(fatx_emd))
fatx_emd.columns=['Review_ft_Embeded'+str(x) for x in range(300)]

In [ ]:
# 그룹바이 하기위해 달아줌
fatx_df=pd.concat([review[['name','prof']],fatx_emd],axis=1)

In [ ]:
# 강의별 임베딩 평균값 생성
feature=[]

for col_name in (['Review_ft_Embeded'+str(x) for x in range(300)]):
    feature.append(fatx_df.groupby(['name','prof'])[col_name].mean().reset_index())

In [ ]:
# 중복값 제거
fatx_df=fatx_df.iloc[:,:2].drop_duplicates()

In [ ]:
# 평균값과 강의명 병합
for f in feature :
    fatx_df =pd.merge(fatx_df, f, on=['name','prof'],how='left')

In [ ]:
# 임베딩 테이블 완성
fatx_df.dropna(inplace=True)
display(find_similar('문명과세계화의도전','안현상',fatx_df)[:5])
display(find_similar('문명과세계화의도전','안현상',sim_df)[:5])

In [ ]:
display(find_similar('영화작문','이지현',fatx_df)[:5])
display(find_similar('영화작문','이지현',sim_df)[:5])
# 결과가 다름 : 수업 분야가 비슷한것들끼리 조금더 잘 뽑은 느낌 (내 주관 + 내가 알고있는 평가)
# 가중치를 줘서 앙상블하는 것도 괜찮지 않을까?